In [3]:
from surprise.dataset import Dataset

# загружаем встроенный набор данных
data = Dataset.load_builtin(name='ml-100k')

In [4]:
from surprise.model_selection import train_test_split

# разбиваем на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
%%time
from surprise import KNNBasic

# обучаем модель
algo = KNNBasic(
    sim_options={
        'user_based': False
    }
)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 908 ms
